In [7]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from ipywidgets import interactive
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box, VBox,Label
import plotly as ply
from dance_sim_tools import trace_objects,utility
from dance_sim_tools.utility import histogram_draw_to_parameters as htp
from dance_sim_tools.ipywidget_helpers import slider

In [8]:
#To test the function converting the histogram draw to the TGA parameter values is correct, make a 
#four figure plot that looks like
#---------------------------- ----------------------------
#           A1               #           A2               #
#    gui slider for          #       reverse computed     #  
#    parameter values        #       parameter values     #            
#    (list)                  #       (list) from          #
#                            #       histogram in B2      #
#                            #                            #
#----------------------------#----------------------------
#                            #                            #
#           B1               #           B2               #
#    theta vs time           #   histogram generated by   #
#        plot for any set of #     <-- trace              #
#     slider values          #                            #
#                            #                            #
#----------------------------#----------------------------#

In [9]:
def g(theta_0,T1,T2,f1,f2,num_transit_ticks,a11,a12):
    
    #We're plotting the relevant crossing lines for a single transit bin,
    #the bin whose lower edge is given by theta_0.
    
    trace = trace_objects.LinearExpandingSinusoidTrace(theta_0,T1,T2,f1,f2,
                    num_transit_ticks=num_transit_ticks,a11=a11,a12=a12)
    transit_tick_interval = 2*np.pi/num_transit_ticks
    
    x = trace.t
    y = (trace.path-np.pi)%(2*np.pi) 
    y = trace.path
    
    fig = plt.figure(1,figsize=(15,15))#,dpi=200)
    ax1 = plt.subplot(221)
   

    plt.plot(x,y,'o',color='grey',markersize=1,zorder=1)
    plt.rc('text', usetex=True)
    plt.plot(x,trace.envelope,'r')

    
    #Plot cyan dots for the reversal points
    reversal_inds = trace.find_reversal_inds()
    plt.scatter(x[reversal_inds],(y%(2*np.pi))[reversal_inds],color='k',zorder=2)
    
    demo_bin = 3
    
    #testing crossing function
    demo_cross_angle = demo_bin*(2*np.pi/num_transit_ticks)
    transit_inds_upper,transit_inds_lower,cross_sign = trace.crosses_0(demo_cross_angle-transit_tick_interval,
                            demo_cross_angle+2*transit_tick_interval)

    transit_inds_upper = np.squeeze(transit_inds_upper)
    transit_inds_lower = np.squeeze(transit_inds_lower)
    cross_sign = np.squeeze(cross_sign)
    
    
   # plt.scatter(x[transit_inds_lower&(cross_sign==-1)],y[transit_inds_lower&(cross_sign==-1)],color='dodgerblue',zorder=2)
   # plt.scatter(x[transit_inds_lower&(cross_sign==1)],y[transit_inds_lower&(cross_sign==1)],color='lightskyblue',zorder=2)
   # plt.scatter(x[transit_inds_upper&(cross_sign==-1)],y[transit_inds_upper&(cross_sign==-1)],color='hotpink',zorder=2)
   # plt.scatter(x[transit_inds_upper&(cross_sign==1)],y[transit_inds_upper&(cross_sign==1)],color='pink',zorder=2)

    t_stop = T1+T2   
#    left_dots = np.vstack((np.zeros(int(num_transit_ticks)),np.linspace(-np.pi,np.pi-(2*np.pi/num_transit_ticks),int(num_transit_ticks))))
 #   right_dots = np.vstack((t_stop*np.ones(int(num_transit_ticks)),np.linspace(-np.pi,np.pi-(2*np.pi/num_transit_ticks),int(num_transit_ticks))))
    #shape outer_dots is (2,num_spots)

#    whole = np.stack((left_dots,right_dots))

#    plt.plot(whole[:,0,:],whole[:,1,:],color='powderblue');
    plt.ylim([-np.pi,np.pi])
    
    #plt.plot([np.min(x),np.max(x)],[demo_cross_angle+2*transit_tick_interval,demo_cross_angle+2*transit_tick_interval],'g',zorder=1)
    #plt.plot([np.min(x),np.max(x)],[demo_cross_angle+1*transit_tick_interval,demo_cross_angle+1*transit_tick_interval],'g',zorder=1)
    #plt.plot([np.min(x),np.max(x)],[demo_cross_angle,demo_cross_angle],'g',zorder=1)
    #plt.plot([np.min(x),np.max(x)],[demo_cross_angle-transit_tick_interval,demo_cross_angle-transit_tick_interval],'g',zorder=1)
    #plt.axhspan(demo_cross_angle,demo_cross_angle+transit_tick_interval,color='pink',alpha=0.5)
    
    #Plot the reversal points
    reversals_inds,reversals_bin_counts = trace.find_reversals_peaks()
    plt.scatter(x[reversal_inds],y[reversal_inds],color='k',zorder=2)
    
    ax2 = plt.subplot(222)
    y = trace.location_bins_rads[:-1]
    ax2.barh(y,reversals_bin_counts,height=(2*np.pi/num_transit_ticks),align='edge',color='r',
             label='reversals')
    #plt.legend()

    plt.ylim([-np.pi,np.pi])
   # plt.plot(whole[:,0,:],whole[:,1,:],color='powderblue')
    #plt.show()
    
    
    tao,alpha,beta = 3,0.7,0.4
    
    
    a11,a12,T1,T2,f1,f2 = htp(reversals_bin_counts,tao,alpha,beta)
    
    #print('a11',a11)
    #print('a12',a12)
    #print('T1',T1)
    #print('T2',T2)
    #print('f1',f1)
    #print('f2',f2)
    
stheta_0 =  slider('theta_0',0,2*np.pi,0.1,np.radians(0))
sT1 = slider('T1',1,10,0.5,5.)
sT2 = slider('T2',1,10,0.5,3.)
sf1 = slider('f1',0,8,0.05,3)
sf2 = slider('f2',0,8,0.05,3)
snum_transit_ticks = slider('num_transit_ticks',0,50,1,24)
sa11 = slider('a11',0,np.pi,0.1,0.5)
sa12 = slider('a12',0,np.pi,0.1,1.)


In [10]:

items = [
    Box([stheta_0]),
    Box([sf1]),
    Box([sf2]),
    Box([sT1]),
    Box([sT2]),
    Box([snum_transit_ticks]),
    Box([sa11]),
    Box([sa12])
]


ui = Box(items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='50%'
))



param_dict =  {'theta_0': stheta_0,
    'T1':sT1,    
    'T2':sT2,
    'f1':sf1,
    'f2':sf2,
    'num_transit_ticks': snum_transit_ticks,
    'a11': sa11,
    'a12': sa12}

# def f(a, b, c):
#     plt.figure()
#     plt.plot(np.array([a]),np.array([b]))
out = widgets.interactive_output(g, param_dict)
display(ui, out)

Box(children=(Box(children=(FloatSlider(value=0.0, continuous_update=False, description='theta_0', max=6.28318…

Output()

In [5]:
a = widgets.IntSlider()
b = widgets.IntSlider()
c = widgets.IntSlider()
ui = widgets.HBox([a, b, c])
def f(a, b, c):
    plt.figure()
    plt.plot(np.array([a]),np.array([b]))
    
#    print((a, b, c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

display(ui, out)

Output()

In [6]:
ShowMe

NameError: name 'ShowMe' is not defined